## Task description:

Provide product sales predictions in order to help to plan stock level.

### Datasets:

**sales.csv**- Data on product-level weekly sales:

- week_starting_date - first day of the week date in format YYYYMMDD
- product_id - unique id for product
- sales - weekly sales in pieces

**categories.csv**- Data on which categories products are assigned to:
- product_id - unique id for product
- category_id - unique id for category

**traffic.csv**:
- week_starting_date - first day of the week date in format YYYYMMDD
- product_id - unique id for product
- traffic - weekly product displays on website'''

## 'A single model to forecast multiple time series at the same time' approach
- more data -> better predictions
- some products don't have much data

## 1. Import Libraries

In [265]:
%matplotlib inline
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pylab as plt 
import statsmodels.api as sm
import datetime
import itertools
import seaborn as sns
from sklearn.dummy import DummyRegressor
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from matplotlib.pylab import rcParams
from os import listdir
from os.path import isfile, join
from prophet import Prophet
import warnings
warnings.simplefilter("ignore")

from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

rcParams['figure.figsize'] = 10, 6
pd.set_option('display.max_rows', 500)

## 2. Data loading week_nbrand spliting

In [266]:
# loading the data
mypath = 'data/'
df_categories = pd.read_csv(join(mypath, 'categories.csv'), sep = ';')
df_sales = pd.read_csv(join(mypath, 'sales.csv'), sep = ';', names=["week_when_sold", "product_id", "sales"], parse_dates=['week_when_sold'], header = 0) 
df_traffic = pd.read_csv(join(mypath, 'traffic.csv'), sep = ';', names=["week_when_displayed_on_website", "product_id", "traffic"], parse_dates=['week_when_displayed_on_website'], header = 0)

In [267]:
# getting some information about the data 'categories.csv'
df_categories.info()
print('\n')
df_categories.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3135 entries, 0 to 3134
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   product_id   3135 non-null   int64
 1   category_id  3135 non-null   int64
dtypes: int64(2)
memory usage: 49.1 KB




,product_id,category_id
0,1990,0
1,2361,1
2,1085,2
3,3091,3
4,955,4


In [268]:
# getting some information about the data 'sales.csv'
df_sales.info()
print('\n')
df_sales.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105781 entries, 0 to 105780
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   week_when_sold  105781 non-null  datetime64[ns]
 1   product_id      105781 non-null  int64         
 2   sales           105781 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 2.4 MB




,week_when_sold,product_id,sales
0,2019-12-09,1990,1
1,2020-11-23,1990,1
2,2020-12-07,1990,1
3,2019-12-02,1990,1
4,2020-11-09,1990,2


In [269]:
# getting some information about the data 'traffic.csv'
df_traffic.info()
print('\n')
df_traffic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176324 entries, 0 to 176323
Data columns (total 3 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   week_when_displayed_on_website  176324 non-null  datetime64[ns]
 1   product_id                      176324 non-null  int64         
 2   traffic                         176324 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.0 MB




,week_when_displayed_on_website,product_id,traffic
0,2019-01-07,1990,1
1,2019-01-07,2361,7
2,2019-01-07,1085,1
3,2019-01-07,3091,4
4,2019-01-07,955,12


In [270]:
# adding traffic data to the dataset (the same week when sold)
df_data = pd.merge(df_sales, df_traffic[['week_when_displayed_on_website','product_id','traffic']], left_on=['week_when_sold', 'product_id'], right_on=['week_when_displayed_on_website', 'product_id'], how='outer')
df_data.rename(columns = {'traffic':'traffic_when_sold_week'}, inplace = True)

In [271]:
df_data.week_when_sold.fillna(0, inplace=True)
df_data.loc[df_data["week_when_sold"] == 0, "week_when_sold"] = df_data["week_when_displayed_on_website"]
df_data.week_when_sold = df_data.week_when_sold.astype('datetime64')
df_data.sales.fillna(0, inplace=True)
df_data.traffic_when_sold_week.fillna(0, inplace=True)
df_data.drop(['week_when_displayed_on_website'], axis=1, inplace=True)
df_data.head()

,week_when_sold,product_id,sales,traffic_when_sold_week
0,2019-12-09,1990,1.0,1.0
1,2020-11-23,1990,1.0,0.0
2,2020-12-07,1990,1.0,0.0
3,2019-12-02,1990,1.0,1.0
4,2020-11-09,1990,2.0,1.0


In [272]:
# adding week number to the dataset
print(min(df_data.week_when_sold))
print(max(df_data.week_when_sold))

2019-01-07 00:00:00
2020-12-28 00:00:00


In [273]:
times = pd.date_range('2019-01-07', periods=105, freq='7D')
_time_df = pd.DataFrame(list(zip(times, np.arange(len(times)))), columns = ['date','week_nbr'])
_time_df.head()

,date,week_nbr
0,2019-01-07,0
1,2019-01-14,1
2,2019-01-21,2
3,2019-01-28,3
4,2019-02-04,4


In [274]:
df_data = pd.merge(df_data, _time_df, left_on='week_when_sold', right_on='date', how='inner')
df_data.drop(['date', 'week_when_sold'], axis=1, inplace=True)
df_data.head()

,product_id,sales,traffic_when_sold_week,week_nbr
0,1990,1.0,1.0,48
1,2361,4.0,11.0,48
2,3091,4.0,5.0,48
3,1603,16.0,6.0,48
4,1824,1.0,11.0,48


In [275]:
df_data.week_nbr.max()

103

In [276]:
# train/val split; we do have ~100 weeks, hence 80th week is covering ~80% of data
split_point = 80
train = df_data[df_data['week_nbr'] < split_point].copy()
val = df_data[df_data['week_nbr'] >= split_point].copy()

## 3. Preparing for predicting 3 weeks ahead

In [277]:
train = train.sort_values(by="week_nbr")
train.head()

,product_id,sales,traffic_when_sold_week,week_nbr
118174,1994,0.0,3.0,0
117452,2319,6.0,40.0,0
117453,823,3.0,2.0,0
117454,2851,1.0,1.0,0
117455,2726,44.0,62.0,0


In [278]:
train['sales_next_week'] = train.groupby(['product_id'])['sales'].shift(-1)
train['sales_2x_next_week'] = train.groupby(['product_id'])['sales'].shift(-2)
train['sales_3x_next_week'] = train.groupby(['product_id'])['sales'].shift(-3)

val['sales_next_week'] = val.groupby(['product_id'])['sales'].shift(-1)
val['sales_2x_next_week'] = val.groupby(['product_id'])['sales'].shift(-2)
val['sales_3x_next_week'] = val.groupby(['product_id'])['sales'].shift(-3)

val.head()

,product_id,sales,traffic_when_sold_week,week_nbr,sales_next_week,sales_2x_next_week,sales_3x_next_week
1844,1990,1.0,0.0,98,1.0,2.0,1.0
1845,2361,4.0,9.0,98,5.0,4.0,8.0
1846,1085,5.0,5.0,98,0.0,0.0,5.0
1847,3091,6.0,7.0,98,7.0,4.0,4.0
1848,1603,10.0,5.0,98,20.0,2.0,5.0


In [279]:
train = train.dropna()

### 4. Feature engineering

### 4.1. Lags and differences

#### Traffic

In [280]:
train['traffic_previous_week'] = train.groupby(['product_id'])['traffic_when_sold_week'].shift(1)
train['traffic_previous_2x_week'] = train.groupby(['product_id'])['traffic_when_sold_week'].shift(2)
train['traffic_previous_3x_week'] = train.groupby(['product_id'])['traffic_when_sold_week'].shift(3)

train['diff_traffic_previous_week'] = train.groupby(['product_id'])['traffic_when_sold_week'].diff(1)
train['diff_traffic_previous_2x_week'] = train.groupby(['product_id'])['traffic_when_sold_week'].diff(2)
train['diff_traffic_previous_3x_week'] = train.groupby(['product_id'])['traffic_when_sold_week'].diff(3)

train.head()

,product_id,sales,traffic_when_sold_week,week_nbr,sales_next_week,sales_2x_next_week,sales_3x_next_week,traffic_previous_week,traffic_previous_2x_week,traffic_previous_3x_week,diff_traffic_previous_week,diff_traffic_previous_2x_week,diff_traffic_previous_3x_week
118174,1994,0.0,3.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
117452,2319,6.0,40.0,0,6.0,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
117453,823,3.0,2.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
117454,2851,1.0,1.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
117455,2726,44.0,62.0,0,58.0,70.0,48.0,NaN,NaN,NaN,NaN,NaN,NaN


In [281]:
val['traffic_previous_week'] = val.groupby(['product_id'])['traffic_when_sold_week'].shift(1)
val['traffic_previous_2x_week'] = val.groupby(['product_id'])['traffic_when_sold_week'].shift(2)
val['traffic_previous_3x_week'] = val.groupby(['product_id'])['traffic_when_sold_week'].shift(3)

val['diff_traffic_previous_week'] = val.groupby(['product_id'])['traffic_when_sold_week'].diff(1)
val['diff_traffic_previous_2x_week'] = val.groupby(['product_id'])['traffic_when_sold_week'].diff(2)
val['diff_traffic_previous_3x_week'] = val.groupby(['product_id'])['traffic_when_sold_week'].diff(3)

val.head()

,product_id,sales,traffic_when_sold_week,week_nbr,sales_next_week,sales_2x_next_week,sales_3x_next_week,traffic_previous_week,traffic_previous_2x_week,traffic_previous_3x_week,diff_traffic_previous_week,diff_traffic_previous_2x_week,diff_traffic_previous_3x_week
1844,1990,1.0,0.0,98,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1845,2361,4.0,9.0,98,5.0,4.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
1846,1085,5.0,5.0,98,0.0,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
1847,3091,6.0,7.0,98,7.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
1848,1603,10.0,5.0,98,20.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN


#### Sales

In [282]:
train['sales_previous_week'] = train.groupby(['product_id'])['sales'].shift(1)
train['sales_previous_2x_week'] = train.groupby(['product_id'])['sales'].shift(2)
train['sales_previous_3x_week'] = train.groupby(['product_id'])['sales'].shift(3)

train['diff_sales_previous_week'] = train.groupby(['product_id'])['sales'].diff(1)
train['diff_sales_previous_2x_week'] = train.groupby(['product_id'])['sales'].diff(2)
train['diff_sales_previous_3x_week'] = train.groupby(['product_id'])['sales'].diff(3)

train.head()

,product_id,sales,traffic_when_sold_week,week_nbr,sales_next_week,sales_2x_next_week,sales_3x_next_week,traffic_previous_week,traffic_previous_2x_week,traffic_previous_3x_week,diff_traffic_previous_week,diff_traffic_previous_2x_week,diff_traffic_previous_3x_week,sales_previous_week,sales_previous_2x_week,sales_previous_3x_week,diff_sales_previous_week,diff_sales_previous_2x_week,diff_sales_previous_3x_week
118174,1994,0.0,3.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117452,2319,6.0,40.0,0,6.0,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117453,823,3.0,2.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117454,2851,1.0,1.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117455,2726,44.0,62.0,0,58.0,70.0,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [283]:
val['sales_previous_week'] = val.groupby(['product_id'])['sales'].shift(1)
val['sales_previous_2x_week'] = val.groupby(['product_id'])['sales'].shift(2)
val['sales_previous_3x_week'] = val.groupby(['product_id'])['sales'].shift(3)

val['diff_sales_previous_week'] = val.groupby(['product_id'])['sales'].diff(1)
val['diff_sales_previous_2x_week'] = val.groupby(['product_id'])['sales'].diff(2)
val['diff_sales_previous_3x_week'] = val.groupby(['product_id'])['sales'].diff(3)

val.head()

,product_id,sales,traffic_when_sold_week,week_nbr,sales_next_week,sales_2x_next_week,sales_3x_next_week,traffic_previous_week,traffic_previous_2x_week,traffic_previous_3x_week,diff_traffic_previous_week,diff_traffic_previous_2x_week,diff_traffic_previous_3x_week,sales_previous_week,sales_previous_2x_week,sales_previous_3x_week,diff_sales_previous_week,diff_sales_previous_2x_week,diff_sales_previous_3x_week
1844,1990,1.0,0.0,98,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1845,2361,4.0,9.0,98,5.0,4.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1846,1085,5.0,5.0,98,0.0,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1847,3091,6.0,7.0,98,7.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1848,1603,10.0,5.0,98,20.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 4.2. Rolling mean

In [284]:
train['rolling_sales_mean'] = train.groupby(['product_id'])['sales'].rolling(4).mean().reset_index(level=0, drop=True)

In [285]:
val['rolling_sales_mean'] = val.groupby(['product_id'])['sales'].rolling(4).mean().reset_index(level=0, drop=True)

In [286]:
train.dropna(inplace=True)
val.dropna(inplace=True)

## 5. Evaluation metric and the baseline
'As of now planning is done on the basis of last week sales - weekly sales are assumed to stay on same level next week' - aaand it's gonna be our baseline

In [287]:
# evaluation metric
def mape(y, y_hat):
    ape = np.abs((y - y_hat) / y)
    ape[~np.isfinite(ape)] = 1.
    return np.mean(ape)

def wmape(y, y_hat):
    return np.sum(np.abs(y - y_hat)) / np.sum(np.abs(y))

def rmse(y, y_hat):
    return np.sqrt(np.mean(np.square(y - y_hat)))

In [288]:
# baseline - checking on the next week sales
y_pred = train['sales']
y_true = train['sales_next_week']

In [289]:
print('WMAPE: ', wmape(y_true, y_pred))
print('MAPE: ', mape(y_true, y_pred))
print('RMSE: ', rmse(y_true, y_pred))

WMAPE:  0.7070115599353408
MAPE:  0.9234219473405727
RMSE:  51.32803793260019


## 6. Model creation

In [290]:
train.head()

,product_id,sales,traffic_when_sold_week,week_nbr,sales_next_week,sales_2x_next_week,sales_3x_next_week,traffic_previous_week,traffic_previous_2x_week,traffic_previous_3x_week,diff_traffic_previous_week,diff_traffic_previous_2x_week,diff_traffic_previous_3x_week,sales_previous_week,sales_previous_2x_week,sales_previous_3x_week,diff_sales_previous_week,diff_sales_previous_2x_week,diff_sales_previous_3x_week,rolling_sales_mean
115896,2852,7.0,9.0,3,0.0,8.0,0.0,9.0,15.0,17.0,0.0,-6.0,-8.0,0.0,0.0,16.0,7.0,7.0,-9.0,5.75
115897,1174,67.0,14.0,3,48.0,64.0,0.0,16.0,23.0,50.0,-2.0,-9.0,-36.0,18.0,40.0,93.0,49.0,27.0,-26.0,54.50
115898,1682,3.0,1.0,3,0.0,0.0,3.0,2.0,3.0,1.0,-1.0,-2.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,0.75
115899,1709,4.0,6.0,3,16.0,8.0,8.0,8.0,10.0,15.0,-2.0,-4.0,-9.0,0.0,8.0,4.0,4.0,-4.0,0.0,4.00
115900,223,20.0,2.0,3,33.0,21.0,5.0,11.0,14.0,2.0,-9.0,-12.0,0.0,36.0,16.0,0.0,-16.0,4.0,20.0,18.00


In [292]:
features = ['sales', 'traffic_when_sold_week', 'traffic_previous_week', 'traffic_previous_2x_week', 'traffic_previous_3x_week', 'diff_traffic_previous_week', 'diff_traffic_previous_2x_week', 'diff_traffic_previous_3x_week', 'sales_previous_week', 
            'sales_previous_2x_week', 'sales_previous_3x_week', 'diff_sales_previous_week', 'diff_sales_previous_2x_week', 'diff_sales_previous_3x_week', 'rolling_sales_mean', 'week_nbr', 'product_id']

In [293]:
X_train = train[features]
y_train = train[['sales_next_week', 'sales_2x_next_week', 'sales_3x_next_week']]

model = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)

## 7. Model evaluation

In [295]:
X_val = val[features]
y_val = val[['sales_next_week', 'sales_2x_next_week', 'sales_3x_next_week']]

pred = model.predict(X_val)

In [296]:
mape(y_val, pred)

sales_next_week       1.060625
sales_2x_next_week    1.154071
sales_3x_next_week    1.180685
dtype: float64

In [297]:
wmape(y_val, pred)

sales_next_week       0.922555
sales_2x_next_week    1.032831
sales_3x_next_week    1.090108
dtype: float64

In [298]:
rmse(y_val, pred)

sales_next_week       71.439503
sales_2x_next_week    75.959629
sales_3x_next_week    77.767114
dtype: float64